In [1]:
#autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [3]:
import warcraftlogs
from warcraftlogs.constants import TOKEN_DIR

from warcraftlogs import WarcraftLogsClient

client = WarcraftLogsClient(token_dir=TOKEN_DIR)

In [288]:
warcraftlogs_url = "https://www.warcraftlogs.com/reports/vWLfFkZ71V3tQAnb?fight=39&type=damage-done&source=12"
warcraftlogs_url = "https://www.warcraftlogs.com/reports/vWLfFkZ71V3tQAnb?fight=28&type=damage-done&source=181"
warcraftlogs_url = "https://www.warcraftlogs.com/reports/vWLfFkZ71V3tQAnb?fight=28&type=damage-done&source=11"
warcraftlogs_url = "https://www.warcraftlogs.com/reports/vWLfFkZ71V3tQAnb?fight=28&type=damage-done&source=9"

# extract report code, fight id, and source id from url
# report code just the last part of the url
REPORT_CODE = re.search(r'/reports/(\w+)', warcraftlogs_url).group(1)
# fight id is the last part of the url
FIGHT_ID = re.search(r'fight=(\d+)', warcraftlogs_url).group(1)
# source id is the last part of the url
SOURCE_ID = re.search(r'source=(\d+)', warcraftlogs_url).group(1)

# convert to int
SOURCE_ID = int(SOURCE_ID)
FIGHT_ID = int(FIGHT_ID)

SOURCE_ID, FIGHT_ID, REPORT_CODE


(9, 28, 'vWLfFkZ71V3tQAnb')

In [289]:
# roll of bone buff not enough, ring not enough stats
# not enough vanishes 

In [290]:
from warcraftlogs.query.tables import get_table_data

damage_done_table_data_query = get_table_data(REPORT_CODE, FIGHT_ID, SOURCE_ID,
                                  data_type="DamageDone")
damage_done_table_data_query

buff_table_data_query = get_table_data(REPORT_CODE, FIGHT_ID, SOURCE_ID,
                                  data_type="Buffs")
buff_table_data_query

'\n    query {\n        reportData {\n            report(code: "vWLfFkZ71V3tQAnb") {\n                table(\n                fightIDs: [28], dataType: Buffs, sourceID: 9\n                )\n            }\n        }\n    }\n    '

In [291]:
print(damage_done_table_data_query)


    query {
        reportData {
            report(code: "vWLfFkZ71V3tQAnb") {
                table(
                fightIDs: [28], dataType: DamageDone, sourceID: 9
                )
            }
        }
    }
    


In [292]:
player_damage_done_table_data_resp = client.query_public_api(damage_done_table_data_query)
player_buff_table_data_resp = client.query_public_api(buff_table_data_query) 

In [293]:
from warcraftlogs.client import WarcraftLogsClient
from warcraftlogs.query.player_analysis import *
from warcraftlogs.query.ranking import generate_ranking_query_from_player_and_fight

In [294]:
player = get_player_details(client, report_code=REPORT_CODE, fight_id=FIGHT_ID, source_id=SOURCE_ID)

In [295]:
player

PlayerDetails(name='Againzt', id=9, spec_name='Marksmanship', role='dps', class_name='Hunter', item_level=663, bracket=11)

In [296]:
fight = get_fight_info(client, report_code=REPORT_CODE, fight_id=FIGHT_ID)

In [297]:
query, variables = generate_ranking_query_from_player_and_fight(player, fight['fights'][0])
ranking_resp = client.query_public_api(query, variables)

In [298]:
similar_player_rankings =ranking_resp['data']['worldData']['encounter']['characterRankings']['rankings']

In [299]:
similar_player_ranking = similar_player_rankings[0]
similar_player_ranking

{'name': '阿玛忒啦斯',
 'class': 'Hunter',
 'spec': 'Marksmanship',
 'amount': 3426885.3020874,
 'hardModeLevel': 0,
 'duration': 348376,
 'startTime': 1743681145829,
 'report': {'code': 'Xf2MkJRALhnD849Z',
  'fightID': 33,
  'startTime': 1743677262667},
 'server': {'id': 750, 'name': '燃烧之刃', 'region': 'CN'},
 'bracketData': 664,
 'faction': 0,
 'size': 29}

In [300]:
similar_player_ranking_player_details =get_player_details(client, report_code=similar_player_ranking['report']['code'],
                    fight_id=similar_player_ranking['report']['fightID'])

In [301]:
# find the id of the player in the similar_player_ranking_player_details
def find_player_id_from_name(fight_player_details: list,
                             player_name: str):
    """
    fight_player_details is a list of dictionaries, each dictionary is a role and the player details
    """
    for role, player_details in fight_player_details.items():
        for player_detail in player_details:
            if player_detail['name'] == player_name:
                return player_detail['id']
    return None

similar_player_ranking_player_id = find_player_id_from_name(similar_player_ranking_player_details['data']['playerDetails'], similar_player_ranking['name'])
similar_player_ranking_player_id

261

In [302]:
from tqdm import tqdm
similar_player_report_info = []
for ranking in tqdm(similar_player_rankings[:10]):
    report_code = ranking['report']['code']
    fight_id = ranking['report']['fightID']
    ranking_player_details = get_player_details(client, report_code=report_code, fight_id=fight_id)
    player_source_id = find_player_id_from_name(ranking_player_details['data']['playerDetails'], ranking['name'])
    similar_player_report_info.append({
        'report_code': ranking['report']['code'],
        'fight_id': ranking['report']['fightID'],
        'player_source_id': player_source_id
    })



100%|██████████| 10/10 [00:05<00:00,  1.93it/s]


In [303]:
top_player_buff_info_lst = []
top_player_dmg_info_lst = []


for report_info in tqdm(similar_player_report_info[:10]):
    buff_table_data_query = get_table_data(report_info['report_code'], report_info['fight_id'], report_info['player_source_id'],
                                          data_type="Buffs", viewOptions=16)
    buff_table_data_resp = client.query_public_api(buff_table_data_query)
    top_player_buff_info_lst.append(buff_table_data_resp['data']['reportData']['report']['table']['data'])

    damage_done_table_data_query = get_table_data(report_info['report_code'], report_info['fight_id'], report_info['player_source_id'],
                                          data_type="DamageDone")
    damage_done_table_data_resp = client.query_public_api(damage_done_table_data_query)
    top_player_dmg_info_lst.append(damage_done_table_data_resp['data']['reportData']['report']['table']['data'])




100%|██████████| 10/10 [00:12<00:00,  1.20s/it]


In [304]:
similar_player_rankings[0]

{'name': '阿玛忒啦斯',
 'class': 'Hunter',
 'spec': 'Marksmanship',
 'amount': 3426885.3020874,
 'hardModeLevel': 0,
 'duration': 348376,
 'startTime': 1743681145829,
 'report': {'code': 'Xf2MkJRALhnD849Z',
  'fightID': 33,
  'startTime': 1743677262667},
 'server': {'id': 750, 'name': '燃烧之刃', 'region': 'CN'},
 'bracketData': 664,
 'faction': 0,
 'size': 29}

In [305]:
similar_player_report_info

[{'report_code': 'Xf2MkJRALhnD849Z', 'fight_id': 33, 'player_source_id': 261},
 {'report_code': 'ANGPKQXYpVWgmF73', 'fight_id': 13, 'player_source_id': 20},
 {'report_code': 'a:YHkLAtpGJxwPKDmr',
  'fight_id': 50,
  'player_source_id': None},
 {'report_code': 'r8Qfch2BNnLz1ZXa', 'fight_id': 73, 'player_source_id': 1587},
 {'report_code': 'krMzvfh7d2VQmjpx', 'fight_id': 29, 'player_source_id': 16},
 {'report_code': '61DcKnabhxw9BFTm', 'fight_id': 39, 'player_source_id': 167},
 {'report_code': '6VWZFdJv7B8NxKbk', 'fight_id': 30, 'player_source_id': 15},
 {'report_code': 'wVRYfJzX2a6FjTPW', 'fight_id': 10, 'player_source_id': 24},
 {'report_code': 'BvJt8bNy2g4GrPVZ', 'fight_id': 27, 'player_source_id': 5},
 {'report_code': 'z9FDpZBHtfCbmyha', 'fight_id': 36, 'player_source_id': 13}]

In [306]:
from warcraftlogs.query.events import get_buff_info_df, get_damage_info_df
from warcraftlogs.analytics.compare import compare_damage_info, compare_buff_uptime

In [307]:
top_player_buff_info_lst_clean =list(map(get_buff_info_df, top_player_buff_info_lst))
top_player_damage_info_lst_clean = list(map(get_damage_info_df, top_player_dmg_info_lst))

# remove empty dataframes
top_player_buff_info_lst_clean = [df for df in top_player_buff_info_lst_clean if not df.empty]
top_player_damage_info_lst_clean = [df for df in top_player_damage_info_lst_clean if not df.empty]

KeyError: 'totalTime' not found in damage_data


In [308]:
player_buff_info_df = get_buff_info_df(player_buff_table_data_resp['data']['reportData']['report']['table']['data'])
player_damage_info_df = get_damage_info_df(player_damage_done_table_data_resp['data']['reportData']['report']['table']['data'])

In [309]:
similar_player_rankings[0]

{'name': '阿玛忒啦斯',
 'class': 'Hunter',
 'spec': 'Marksmanship',
 'amount': 3426885.3020874,
 'hardModeLevel': 0,
 'duration': 348376,
 'startTime': 1743681145829,
 'report': {'code': 'Xf2MkJRALhnD849Z',
  'fightID': 33,
  'startTime': 1743677262667},
 'server': {'id': 750, 'name': '燃烧之刃', 'region': 'CN'},
 'bracketData': 664,
 'faction': 0,
 'size': 29}

In [310]:
player_damage_info_df[['name','dps', 'hitCount', 'critHitCount', 'crit_pct', 'uses','hit_per_minute',]].\
    sort_values("dps", ascending=False)

,name,dps,hitCount,critHitCount,crit_pct,uses,hit_per_minute
6,Black Arrow,629839.0,234,142,61.0,49.0,38.19
7,Aimed Shot,226376.0,40,21,52.0,40.0,6.53
9,Explosive Shot,174291.0,21,10,48.0,8.0,3.43
0,Master Marksman,137802.0,0,0,NaN,NaN,0.00
4,Rapid Fire,126928.0,161,96,60.0,18.0,26.28
10,Arcane Shot,110004.0,47,27,57.0,49.0,7.67
5,Bleak Arrows,105609.0,74,48,65.0,NaN,12.08
2,Dark Hounds,53460.0,80,38,48.0,NaN,13.06
3,Authority of Radiant Power,23944.0,17,9,53.0,NaN,2.77
1,Steady Shot,17240.0,35,17,49.0,35.0,5.71


In [311]:
compare_damage_info(player_damage_info_df, top_player_damage_info_lst_clean[5]).\
    sort_values('dps_diff', ascending=False)[['ability','dps_base','dps_compare','dps_diff']].\
    style.background_gradient(cmap='RdYlGn', subset=['dps_diff'])

,ability,dps_base,dps_compare,dps_diff
0,Aimed Shot,226376.000000,939594.000000,713218.000000
3,Black Arrow,629839.000000,1096395.000000,466556.000000
7,Master Marksman,137802.000000,297477.000000,159675.000000
8,Rapid Fire,126928.000000,235314.000000,108386.000000
5,Dark Hounds,53460.000000,127243.000000,73783.000000
1,Arcane Shot,110004.000000,167539.000000,57535.000000
4,Bleak Arrows,105609.000000,116985.000000,11376.000000
9,Steady Shot,17240.000000,24480.000000,7240.000000
2,Authority of Radiant Power,23944.000000,21205.000000,-2739.000000
6,Explosive Shot,174291.000000,169536.000000,-4755.000000


In [312]:
counter = 0

In [321]:
counter+=1
compare_damage_info(player_damage_info_df, top_player_damage_info_lst_clean[counter]).\
    sort_values('hit_per_minute_diff', ascending=False)[['ability', 'hit_per_minute_base', 'hit_per_minute_compare', 'hit_per_minute_diff']].\
    query("hit_per_minute_base > 1").\
    style.background_gradient(cmap='RdYlGn', subset=['hit_per_minute_diff'])



,ability,hit_per_minute_base,hit_per_minute_compare,hit_per_minute_diff
8,Rapid Fire,26.300000,56.600000,30.400000
3,Black Arrow,38.200000,57.900000,19.700000
5,Dark Hounds,13.100000,25.700000,12.600000
0,Aimed Shot,6.500000,17.900000,11.300000
4,Bleak Arrows,12.100000,14.800000,2.800000
9,Steady Shot,5.700000,8.300000,2.600000
1,Arcane Shot,7.700000,9.600000,1.900000
6,Explosive Shot,3.400000,4.000000,0.600000
2,Authority of Radiant Power,2.800000,2.300000,-0.500000


In [314]:
compare_damage_info(player_damage_info_df, top_player_damage_info_lst_clean[0]).\
    sort_values('dps_diff', ascending=False).\
    style.background_gradient(cmap='RdYlGn', subset=['dps_diff','hit_per_minute_diff'])

,ability,dps_base,dps_compare,dps_diff,dps_diff_pct,hit_per_minute_base,hit_per_minute_compare,hit_per_minute_diff,hit_per_minute_diff_pct
0,Aimed Shot,226376.000000,785866.000000,559490.000000,247.200000,6.500000,22.400000,15.900000,242.900000
2,Black Arrow,629839.000000,1080200.000000,450361.000000,71.500000,38.200000,42.200000,4.000000,10.500000
5,Explosive Shot,174291.000000,392381.000000,218090.000000,125.100000,3.400000,6.500000,3.100000,90.700000
7,Rapid Fire,126928.000000,290157.000000,163229.000000,128.600000,26.300000,84.700000,58.500000,222.500000
6,Master Marksman,137802.000000,290434.000000,152632.000000,110.800000,0.000000,0.000000,0.000000,nan
1,Arcane Shot,110004.000000,200022.000000,90018.000000,81.800000,7.700000,12.100000,4.400000,57.200000
4,Dark Hounds,53460.000000,125819.000000,72359.000000,135.400000,13.100000,34.600000,21.600000,165.100000
3,Bleak Arrows,105609.000000,122188.000000,16579.000000,15.700000,12.100000,14.600000,2.600000,21.200000
8,Steady Shot,17240.000000,16395.000000,-845.000000,-4.900000,5.700000,6.000000,0.300000,5.600000


In [322]:
# color the up_time_pct_diff column by the magnitude of the difference
compare_buff_uptime(player_buff_info_df, top_player_buff_info_lst_clean[0]).\
    sort_values(by='up_time_pct_diff', ascending=True).head(20).\
    style.background_gradient(cmap='RdYlGn', subset=['abs_up_time_pct_diff'])

,name,up_time_pct_1,totalUses_1,type_1,guid,up_time_pct_2,totalUses_2,type_2,up_time_pct_diff,total_uses_diff,abs_up_time_pct_diff
17,Moving Target,0.600000,32,1,474293,0.780000,82,1,-0.180000,-50,0.180000
2,Bulletstorm,0.650000,161,1,389020,0.780000,325,1,-0.130000,-164,0.130000
15,Jackpot!,0.410000,22,1,1218033,0.530000,24,1,-0.120000,-2,0.120000
12,Trueshot,0.160000,3,1,288613,0.240000,4,1,-0.080000,-1,0.080000
6,Withering Fire,0.160000,3,32,466991,0.240000,4,32,-0.080000,-1,0.080000
21,Rejuvenating Wind,0.070000,4,8,385540,0.140000,6,8,-0.070000,-2,0.070000
25,Streamline,0.870000,100,1,342076,0.910000,201,1,-0.040000,-101,0.040000
0,Survival of the Fittest,0.050000,6,1,264735,0.090000,9,1,-0.040000,-3,0.040000
9,Explosive Adrenaline,0.490000,6,1,1218713,0.520000,6,1,-0.030000,0,0.030000
8,Tempered Potion,0.160000,2,1,431932,0.170000,2,1,-0.010000,0,0.010000


In [316]:
# color the up_time_pct_diff column by the magnitude of the difference
compare_buff_uptime(player_buff_info_df, top_player_buff_info_lst_clean[0]).\
    sort_values(by='abs_up_time_pct_diff', ascending=False).head(10).\
    style.background_gradient(cmap='RdYlGn', subset=['abs_up_time_pct_diff'])

,name,up_time_pct_1,totalUses_1,type_1,guid,up_time_pct_2,totalUses_2,type_2,up_time_pct_diff,total_uses_diff,abs_up_time_pct_diff
17,Moving Target,0.600000,32,1,474293,0.780000,82,1,-0.180000,-50,0.180000
10,Razor Fragments,0.320000,40,1,388998,0.180000,50,1,0.140000,-10,0.140000
2,Bulletstorm,0.650000,161,1,389020,0.780000,325,1,-0.130000,-164,0.130000
24,Deathblow,0.310000,40,1,378770,0.180000,50,1,0.130000,-10,0.130000
15,Jackpot!,0.410000,22,1,1218033,0.530000,24,1,-0.120000,-2,0.120000
3,Trailblazer,0.100000,16,1,231390,0.000000,2,1,0.100000,14,0.100000
13,Dawnthread Lining,0.830000,8,1,457666,0.730000,8,1,0.100000,0,0.100000
18,Posthaste,0.100000,9,1,118922,0.020000,2,1,0.080000,7,0.080000
12,Trueshot,0.160000,3,1,288613,0.240000,4,1,-0.080000,-1,0.080000
6,Withering Fire,0.160000,3,32,466991,0.240000,4,32,-0.080000,-1,0.080000
